# Extract Transform Load (ETL)

Extract:
Extract the Electric Vehicle data for Washington State for further analysis. Extracted the Data for the Electric Vehicle Title and Registration Activity for Washington State from data.wa.gov  in the form of CSV file. To enrich the data we got in the  form of csv file we made API call to data.wa.gov and got DOL Vehicle ID ,Clean Alternative Fuel Vehicle (CAFV) Eligibility and Electric Utility for Washington State. 

Clean Alternative Fuel Vehicle (CAFV) Eligibility- This categorizes vehicle as Clean Alternative Fuel Vehicles (CAFVs) based on the fuel requirement and electric-only range requirement in House Bill 2042 as passed in the 2019 legislative session.

Electric Utility -This is the electric power retail service territories serving the address of the registered vehicle.

DOL Vehicle ID- A unique identification number for each vehicle present in Transactions dataset. Transactions done on the same vehicle will have the same DOL Vehicle ID.

Transform:
Used the pandas to convert the CSV file data and API call data into dataframe and performed following steps to clean the data:
-Dropped the columns which we don’t require for further analysis.
-Filtered the data by transaction type is equal to Original Registration and transaction year is greater or equal to 2020.
-Outer Joined the two dataframe using DOL Vehicle ID.

Load:
Loaded data  into Mongo DB


In [1]:
import pandas as pd
import requests
from pprint import pprint
import pymongo

In [2]:
df = pd.read_csv('Resources/Electric_Vehicle_Title_and_Registration_Activity.csv')
#df.head()
df.columns
#df.describe()

Index(['Clean Alternative Fuel Vehicle Type', 'VIN (1-10)', 'Model Year',
       'Make', 'Model', 'New or Used Vehicle', 'Sale Price',
       'DOL Transaction Date', 'Transaction Type', 'Transaction Year',
       'Electric Vehicle Fee Paid', 'County', 'City', 'Postal Code',
       'Electric Range', 'Base MSRP', '2015 HB 2778 Exemption Eligibility',
       'Sale Date', 'Vehicle Primary Use', 'State of Residence',
       'DOL Vehicle ID', 'Legislative District',
       '2019 HB 2042 Clean Alternative Fuel Vehicle (CAFV) Eligibility',
       'Meets 2019 HB 2042 Electric Range Requirement',
       'Meets 2019 HB 2042 Sale Date Requirement',
       'Meets 2019 HB 2042 Sale Price/Value Requirement', 'Odometer Reading',
       'Odometer Code', 'Transportation Electrification Fee Paid',
       'Hybrid Vehicle Electrification Fee Paid', '2020 Census Tract'],
      dtype='object')

In [3]:
# selecting all rows from given dataframe in which transaction type is equal to  Original Registration 
#and transaction year is greater or equal to 2020

df = df[(df['Transaction Type'] == 'Original Registration') & (df['Transaction Year'] >= 2020)]
df

,Clean Alternative Fuel Vehicle Type,VIN (1-10),Model Year,Make,Model,New or Used Vehicle,Sale Price,DOL Transaction Date,Transaction Type,Transaction Year,...,Legislative District,2019 HB 2042 Clean Alternative Fuel Vehicle (CAFV) Eligibility,Meets 2019 HB 2042 Electric Range Requirement,Meets 2019 HB 2042 Sale Date Requirement,Meets 2019 HB 2042 Sale Price/Value Requirement,Odometer Reading,Odometer Code,Transportation Electrification Fee Paid,Hybrid Vehicle Electrification Fee Paid,2020 Census Tract
14,Battery Electric Vehicle (BEV),5YJ3E1EC9N,2022,TESLA,Model 3,New,0,October 28 2022,Original Registration,2022,...,41.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303302e+10
36,Battery Electric Vehicle (BEV),1N4CZ1CV4P,2023,NISSAN,Leaf,New,0,October 20 2022,Original Registration,2022,...,47.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303303e+10
40,Battery Electric Vehicle (BEV),5YJ3E1EB0N,2022,TESLA,Model 3,New,0,October 11 2022,Original Registration,2022,...,45.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303302e+10
53,Battery Electric Vehicle (BEV),1G1FY6S03L,2020,CHEVROLET,Bolt EV,New,0,October 19 2020,Original Registration,2020,...,1.0,"TRANSACTION NOT ELIGIBLE: Non-sale, registrati...",True,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303302e+10
77,Battery Electric Vehicle (BEV),YV4ED3UR6N,2022,VOLVO,XC40,New,0,November 24 2021,Original Registration,2021,...,48.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303302e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617674,Battery Electric Vehicle (BEV),1N4BZ1BV9N,2022,NISSAN,Leaf,New,0,December 13 2021,Original Registration,2021,...,9.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.302102e+10
617677,Battery Electric Vehicle (BEV),WVGGNPE20N,2022,VOLKSWAGEN,ID.4,New,0,September 07 2022,Original Registration,2022,...,2.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.305307e+10
617679,Battery Electric Vehicle (BEV),5YJ3E1EBXN,2022,TESLA,Model 3,New,0,February 10 2022,Original Registration,2022,...,38.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.306104e+10
617687,Battery Electric Vehicle (BEV),5YJ3E1EA0N,2022,TESLA,Model 3,New,0,February 18 2022,Original Registration,2022,...,1.0,ERROR: No battery range; TRANSACTION NOT ELIGI...,False,False,False,0,Odometer reading is not collected at time of r...,No,No,5.303302e+10


In [4]:
# dropped the columns to clean up the data.
clean_df = df.drop(['New or Used Vehicle','Sale Price','Electric Vehicle Fee Paid','2015 HB 2778 Exemption Eligibility','Base MSRP','Sale Date','2019 HB 2042 Clean Alternative Fuel Vehicle (CAFV) Eligibility','Meets 2019 HB 2042 Electric Range Requirement','Meets 2019 HB 2042 Sale Date Requirement','Meets 2019 HB 2042 Sale Price/Value Requirement','Hybrid Vehicle Electrification Fee Paid','2020 Census Tract', 'Odometer Reading',
       'Odometer Code'], axis=1)
clean_df

,Clean Alternative Fuel Vehicle Type,VIN (1-10),Model Year,Make,Model,DOL Transaction Date,Transaction Type,Transaction Year,County,City,Postal Code,Electric Range,Vehicle Primary Use,State of Residence,DOL Vehicle ID,Legislative District,Transportation Electrification Fee Paid
14,Battery Electric Vehicle (BEV),5YJ3E1EC9N,2022,TESLA,Model 3,October 28 2022,Original Registration,2022,King,BELLEVUE,98006.0,0,Passenger,WA,219181686,41.0,No
36,Battery Electric Vehicle (BEV),1N4CZ1CV4P,2023,NISSAN,Leaf,October 20 2022,Original Registration,2022,King,AUBURN,98092.0,0,Passenger,WA,224487602,47.0,No
40,Battery Electric Vehicle (BEV),5YJ3E1EB0N,2022,TESLA,Model 3,October 11 2022,Original Registration,2022,King,WOODINVILLE,98072.0,0,Passenger,WA,217970987,45.0,No
53,Battery Electric Vehicle (BEV),1G1FY6S03L,2020,CHEVROLET,Bolt EV,October 19 2020,Original Registration,2020,King,KIRKLAND,98034.0,259,Passenger,WA,125985527,1.0,No
77,Battery Electric Vehicle (BEV),YV4ED3UR6N,2022,VOLVO,XC40,November 24 2021,Original Registration,2021,King,KIRKLAND,98033.0,0,Passenger,WA,180098852,48.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617674,Battery Electric Vehicle (BEV),1N4BZ1BV9N,2022,NISSAN,Leaf,December 13 2021,Original Registration,2021,Franklin,PASCO,99301.0,0,Passenger,WA,186000923,9.0,No
617677,Battery Electric Vehicle (BEV),WVGGNPE20N,2022,VOLKSWAGEN,ID.4,September 07 2022,Original Registration,2022,Pierce,PUYALLUP,98375.0,0,Passenger,WA,212075832,2.0,No
617679,Battery Electric Vehicle (BEV),5YJ3E1EBXN,2022,TESLA,Model 3,February 10 2022,Original Registration,2022,Snohomish,EVERETT,98201.0,0,Passenger,WA,190060069,38.0,No
617687,Battery Electric Vehicle (BEV),5YJ3E1EA0N,2022,TESLA,Model 3,February 18 2022,Original Registration,2022,King,KIRKLAND,98034.0,0,Passenger,WA,192455651,1.0,No


In [5]:

# left_df = clean_df

In [6]:
# https://data.wa.gov/resource/f6w7-q2d2.json?dol_vehicle_id=1908966
url = "https://data.wa.gov/resource/f6w7-q2d2.json?"
def get_cafv_utility(limit):
    query_url = 'https://data.wa.gov/resource/f6w7-q2d2.json?$select=dol_vehicle_id,cafv_type,electric_utility&$limit=' + str(limit)
    response = requests.get(query_url)
    if not response.ok:
        return None
        
    response = response.json()
    print(type(response))
    return pd.DataFrame(response)

df = get_cafv_utility(110000)
# df = df[df['dol_vehicle_id'].isin(['200589147','219181686', '224487602'])]
df['dol_vehicle_id'] = df['dol_vehicle_id'].astype(int)

<class 'list'>


In [7]:
merged_df=pd.merge(clean_df,df,left_on='DOL Vehicle ID', right_on='dol_vehicle_id',how='left')

In [8]:
#Uploading The Pandas DataFrame to MongoDB
# Convert dataframe to dictinary
merged_dict=merged_df.to_dict("records")
merged_dict

[{'Clean Alternative Fuel Vehicle Type': 'Battery Electric Vehicle (BEV)',
  'VIN (1-10)': '5YJ3E1EC9N',
  'Model Year': 2022,
  'Make': 'TESLA',
  'Model': 'Model 3',
  'DOL Transaction Date': 'October 28 2022',
  'Transaction Type': 'Original Registration',
  'Transaction Year': 2022,
  'County': 'King',
  'City': 'BELLEVUE',
  'Postal Code': 98006.0,
  'Electric Range': 0,
  'Vehicle Primary Use': 'Passenger',
  'State of Residence': 'WA',
  'DOL Vehicle ID': 219181686,
  'Legislative District': 41.0,
  'Transportation Electrification Fee Paid': 'No',
  'dol_vehicle_id': 219181686.0,
  'cafv_type': 'Eligibility unknown as battery range has not been researched',
  'electric_utility': 'PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)'},
 {'Clean Alternative Fuel Vehicle Type': 'Battery Electric Vehicle (BEV)',
  'VIN (1-10)': '1N4CZ1CV4P',
  'Model Year': 2023,
  'Make': 'NISSAN',
  'Model': 'Leaf',
  'DOL Transaction Date': 'October 20 2022',
  'Transaction Type': 'Original Registration

In [9]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


# Define the 'classDB' database in Mongo
db = client.ev_DB

db.titles.drop()

#collection = db['<<INSERT NAME OF COLLECTION>>']

collection = db['titles']

# Insert collection
collection.insert_many(merged_dict)